# Distribution method

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, set_seed
from datasets import load_dataset
import pandas as pd
import torch
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
set_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [47]:
xx = torch.load("/kaggle/input/raw-qwen05-predict-200/raw_qwen05_predict_massive200_selected_v.pt", weights_only=False)

if (xx < 0).any():
    print("⚠️ Warning: The tensor contains negative values!")
else:
    print("✅ No negative values found.")


<ipython-input-47-679e162b6ddf>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  xx = torch.load("/kaggle/input/raw-qwen05-predict-200/raw_qwen05_predict_massive200_selected

⚠️ Warning: The tensor contains negative values!


In [3]:
import torch
# full_languages_raw_values = torch.abs(torch.randn(2, 3, 4))
full_languages_raw_values = torch.load("/kaggle/input/raw-qwen05-predict-200/raw_qwen05_predict_massive200_selected_v.pt", weights_only=False)
full_languages_raw_values.shape

torch.Size([52, 2, 200, 116736])

In [4]:
full_languages_raw_values=full_languages_raw_values[:,0,:,:]
full_languages_raw_values.shape

torch.Size([52, 200, 116736])

i have a tensor of shape 5,2,3,and i want to calculate kl divergence for each i in dim -1, and make kl divergence matrix of 5 by 5 where each cell is the kl divergence of [j,:i] and [k,:,i] where j and k is element of the first dimension 

In [94]:
import torch
import torch.nn.functional as F
from tqdm import tqdm

# Example tensor of shape (5, 2, 3)
p = full_languages_raw_values
p = torch.clamp(p, min=0)
p /= p.sum(dim=1, keepdim=True)  # Normalize to make valid probability distributions

# KL divergence matrix (5x5)
num_lang, num_rows, num_neurons = full_languages_raw_values.shape
kl_matrix = torch.zeros(num_lang,num_lang,num_neurons)  # One KL divergence per i (dim=-1)

for i in tqdm(range(num_neurons)):  # Iterate over last dimension
    for j in range(num_lang):  # Iterate over first dimension
        for k in range(num_lang):  # Iterate over first dimension
            kl_matrix[j, k, i] = F.kl_div(
                p[j, :, i].log(),  # log of distribution p
                p[k, :, i],  # distribution q
                reduction='sum'
            )

# Reduce over last dimension if needed
# kl_final = kl_matrix.sum(dim=-1)  # Shape (5,5)
kl_matrix[torch.abs(kl_matrix) < 1e-6] = 0

print(kl_matrix)


KeyboardInterrupt: 

In [ ]:
import torch
import torch.nn.functional as F

# Example tensor of shape (5, 2, 3)

p = full_languages_raw_values
num_lang, num_rows, num_neurons = full_languages_raw_values.shape

del full_languages_raw_values
p = torch.clamp(p, min=0)
p /= p.sum(dim=1, keepdim=True)  # Normalize along dim=1 to ensure valid probabilities
p = torch.clamp(p, min=1e-8)


kl_matrix1 = torch.zeros(num_lang,num_rows,num_neurons)  # One KL divergence per i (dim=-1)

# Expand dimensions for broadcasting
p1 = p[:, None, :, :]  # Shape (5, 1, 2, 3) → For j
p2 = p[None, :, :, :]  # Shape (1, 5, 2, 3) → For k

# p1
# Compute KL divergence for each i (dim=-1)
kl_matrix1 = F.kl_div(p1.log(), p2, reduction="none").sum(dim=2)  # Shape (5, 5, 3)
kl_matrix1[torch.abs(kl_matrix) < 1e-6] = 0

print(kl_matrix1)  # Should output: torch.Size([5, 5, 3])


OOM remedy

In [5]:
import torch
import torch.nn.functional as F
from tqdm import tqdm 
# Example tensor (Replace with your actual tensor)
# p = torch.randn(50, 2, 3)  # Example shape (num_lang=50, num_rows=2, num_neurons=3)
p = full_languages_raw_values
num_lang, num_rows, num_neurons = p.shape

del full_languages_raw_values
num_lang, num_rows, num_neurons = p.shape

# Normalize
p = torch.clamp(p, min=0)
p /= p.sum(dim=1, keepdim=True)  # Normalize along dim=1
p = torch.clamp(p, min=1e-8)  # Prevent log(0) issues

# Initialize KL matrix

kl_matrix = torch.zeros(num_lang, num_lang, num_neurons, device=p.device)  

# Compute KL divergence pairwise to save memory
for i in tqdm(range(num_lang)):
    p1 = p[i].unsqueeze(0)  # Shape: (1, num_rows, num_neurons)
    kl_matrix[i] = F.kl_div(p1.log(), p, reduction="none").sum(dim=1)  # Reduce dim=1 (num_rows)

# Set small values to zero
kl_matrix[torch.abs(kl_matrix) < 1e-6] = 0

print(kl_matrix.shape)  # Expected shape: (num_lang, num_lang, num_neurons)


100%|██████████| 52/52 [11:37<00:00, 13.42s/it]


torch.Size([52, 52, 116736])


In [ ]:
kl_matrix == kl_matrix1

In [46]:
for i in range(kl_matrix.size(-1)):
    print(kl_matrix[:,:,i])

tensor([[0.0000, 0.1112],
        [0.0907, 0.0000]])
tensor([[0.0000, 0.2680],
        [0.2860, 0.0000]])
tensor([[0.0000, 0.2477],
        [0.1530, 0.0000]])
tensor([[0.0000, 0.3812],
        [0.3594, 0.0000]])


In [6]:
kl_matrix = kl_matrix.transpose(0, -1)
print(kl_matrix)
kl_matrix.shape

tensor([[[   nan,    nan,    nan,  ...,    nan,    nan,    nan],
         [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
         [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
         ...,
         [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
         [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
         [   nan,    nan,    nan,  ...,    nan,    nan,    nan]],

        [[   nan,    nan,    nan,  ...,    nan,    nan,    nan],
         [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
         [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
         ...,
         [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
         [   nan,    nan,    nan,  ...,    nan,    nan,    nan],
         [   nan,    nan,    nan,  ...,    nan,    nan,    nan]],

        [[0.0000, 0.0511, 0.0627,  ..., 0.0546, 0.0645, 0.0655],
         [0.0545, 0.0000, 0.0215,  ..., 0.0210, 0.0339, 0.0349],
         [0.0633, 0.0221, 0.0000,  ..., 0.0315, 0.0411, 0.

torch.Size([116736, 52, 52])

In [7]:
kl_matrix_clean = kl_matrix
kl_matrix[torch.isinf(kl_matrix)] = 0
kl_matrix[torch.isnan(kl_matrix)] = 0

In [8]:
avgd = kl_matrix.mean(dim=-1)
avgd

tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0635, 0.0284, 0.0404,  ..., 0.0363, 0.0482, 0.0487],
        ...,
        [0.0039, 0.0049, 0.0046,  ..., 0.0047, 0.0065, 0.0072],
        [0.0492, 0.1153, 0.1607,  ..., 0.0554, 0.0709, 0.0639],
        [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]])

In [9]:
torch.max(avgd, dim=-1)

torch.return_types.max(
values=tensor([0.0000, 0.0000, 0.1925,  ..., 0.0107, 1.0828, 0.0000]),
indices=tensor([ 0,  0, 24,  ..., 22, 32,  0]))

In [10]:
threshold = torch.quantile(avgd.float().flatten(), 0.9)  
threshold

tensor(0.0497)

In [17]:
each, index = torch.where(avgd >= threshold)
values = avgd[each, index]
neurons_lang_specific = dict()
lang_specific_neurons = dict()
for i,j in zip(each,index):
    neuron = i.item()
    lang = j.item()
    if neuron in lang_specific_neurons:
        lang_specific_neurons[neuron].append(lang)
    else:
        lang_specific_neurons[neuron] = [lang]
    if lang in neurons_lang_specific:
        neurons_lang_specific[lang].append(neuron)
    else:
        neurons_lang_specific[lang] = [neuron]

In [19]:
torch.save(neurons_lang_specific, "neurons_lang_specific")

In [18]:
len(neurons_lang_specific)

52

In [15]:
torch.save(lang_specific_neurons, "lang_specific_neurons")

## LAPE > 0.7

In [6]:
languages = {
    0: 'Afrikaans (af-ZA)',
    1: 'Amharic (am-ET)',
    2: 'Arabic (ar-SA)',
    3: 'Azerbaijani (az-AZ)',
    4: 'Bengali (bn-BD)',
    5: 'Catalan (ca-ES)',
    6: 'Welsh (cy-GB)',
    7: 'Danish (da-DK)',
    8: 'German (de-DE)',
    9: 'Greek (el-GR)',
    10: 'English (en-US)',
    11: 'Spanish (es-ES)',
    12: 'Persian (fa-IR)',
    13: 'Finnish (fi-FI)',
    14: 'French (fr-FR)',
    15: 'Hebrew (he-IL)',
    16: 'Hindi (hi-IN)',
    17: 'Hungarian (hu-HU)',
    18: 'Armenian (hy-AM)',
    19: 'Indonesian (id-ID)',
    20: 'Icelandic (is-IS)',
    21: 'Italian (it-IT)',
    22: 'Japanese (ja-JP)',
    23: 'Javanese (jv-ID)',
    24: 'Georgian (ka-GE)',
    25: 'Khmer (km-KH)',
    26: 'Kannada (kn-IN)',
    27: 'Korean (ko-KR)',
    28: 'Latvian (lv-LV)',
    29: 'Malayalam (ml-IN)',
    30: 'Mongolian (mn-MN)',
    31: 'Malay (ms-MY)',
    32: 'Burmese (my-MM)',
    33: 'Norwegian Bokmål (nb-NO)',
    34: 'Dutch (nl-NL)',
    35: 'Polish (pl-PL)',
    36: 'Portuguese (pt-PT)',
    37: 'Romanian (ro-RO)',
    38: 'Russian (ru-RU)',
    39: 'Slovenian (sl-SL)',
    40: 'Albanian (sq-AL)',
    41: 'Swedish (sv-SE)',
    42: 'Swahili (sw-KE)',
    43: 'Tamil (ta-IN)',
    44: 'Telugu (te-IN)',
    45: 'Thai (th-TH)',
    46: 'Tagalog (tl-PH)',
    47: 'Turkish (tr-TR)',
    48: 'Urdu (ur-PK)',
    49: 'Vietnamese (vi-VN)',
    50: 'Chinese (Simplified) (zh-CN)',
    51: 'Chinese (Traditional) (zh-TW)'
}

In [ ]:
xx = torch.load("/kaggle/input/overlap-spearman-langs-neurons/lang_overlap_massive_qwen05", weights_only=False)


In [4]:
import torch

for k, v in xx.items():
    xx[k] = xx[k][:5]

In [5]:
xx

{'af-ZA': [(34, 0.8879334836782731),
  (33, 0.8491741348126053),
  (7, 0.8488753020032594),
  (41, 0.8472072125805541),
  (8, 0.8342742823561485)],
 'am-ET': [(26, 0.8261336256465533),
  (29, 0.8200113202498492),
  (25, 0.8128715432916865),
  (32, 0.8027261946360293),
  (44, 0.7928812092144808)],
 'ar-SA': [(15, 0.820893974882431),
  (12, 0.8069446059992754),
  (45, 0.8005463687225443),
  (38, 0.790372300148327),
  (48, 0.7806279454327465)],
 'az-AZ': [(47, 0.8641485639751891),
  (40, 0.826030308369444),
  (28, 0.8157174379687357),
  (20, 0.8112456715293006),
  (17, 0.8100032659615058)],
 'bn-BD': [(16, 0.8993959270553938),
  (43, 0.854619395605349),
  (44, 0.847438123741576),
  (29, 0.8435036724748316),
  (48, 0.8281206256537281)],
 'ca-ES': [(11, 0.8540355674402333),
  (21, 0.848600381226134),
  (36, 0.8442361086930477),
  (37, 0.844236050421409),
  (14, 0.8330458330434538)],
 'cy-GB': [(42, 0.8455797732839397),
  (40, 0.84219128040732),
  (20, 0.8308372278535836),
  (28, 0.818476237

In [7]:
translated_data = {
    key: [(languages[first], second) for first, second in values] 
    for key, values in xx.items()
}

# Print the new dictionary
print(translated_data)

{'af-ZA': [('Dutch (nl-NL)', 0.8879334836782731), ('Norwegian Bokmål (nb-NO)', 0.8491741348126053), ('Danish (da-DK)', 0.8488753020032594), ('Swedish (sv-SE)', 0.8472072125805541), ('German (de-DE)', 0.8342742823561485)], 'am-ET': [('Kannada (kn-IN)', 0.8261336256465533), ('Malayalam (ml-IN)', 0.8200113202498492), ('Khmer (km-KH)', 0.8128715432916865), ('Burmese (my-MM)', 0.8027261946360293), ('Telugu (te-IN)', 0.7928812092144808)], 'ar-SA': [('Hebrew (he-IL)', 0.820893974882431), ('Persian (fa-IR)', 0.8069446059992754), ('Thai (th-TH)', 0.8005463687225443), ('Russian (ru-RU)', 0.790372300148327), ('Urdu (ur-PK)', 0.7806279454327465)], 'az-AZ': [('Turkish (tr-TR)', 0.8641485639751891), ('Albanian (sq-AL)', 0.826030308369444), ('Latvian (lv-LV)', 0.8157174379687357), ('Icelandic (is-IS)', 0.8112456715293006), ('Hungarian (hu-HU)', 0.8100032659615058)], 'bn-BD': [('Hindi (hi-IN)', 0.8993959270553938), ('Tamil (ta-IN)', 0.854619395605349), ('Telugu (te-IN)', 0.847438123741576), ('Malayala

In [11]:
import pandas as pd

# Convert dictionary into a structured list
structured_data = []
for lang, correlations in translated_data.items():
    for related_lang, score in correlations:
        structured_data.append({"Language": lang, "Related Language": related_lang, "Correlation": score})

# Create DataFrame
df = pd.DataFrame(structured_data)

# Sort values for better visualization
df = df.sort_values(by=["Language", "Correlation"], ascending=[True, False])

# Show the first few rows
# print(df.head())df
df.to_csv("language_correlation.csv", index=False)
